In [1]:
import time
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Reddit
from torch_geometric.loader import NeighborLoader

In [2]:
dataset = Reddit(root='/tmp/Reddit')
data = dataset[0]
print(2 * data.num_edges / data.num_nodes) # 平均次数

983.9752065760952


In [3]:
class GCN(torch.nn.Module):
    def __init__(self, in_d, mid_d, out_d):
        super().__init__()
        self.conv1 = GCNConv(in_d, mid_d)
        self.conv2 = GCNConv(mid_d, out_d)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [4]:
def calc_acc(model):
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

In [5]:
model = GCN(dataset.num_node_features, 32, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=1e-4)

In [6]:
def train(epoch):
    model.train()
    start = time.time()
    for epoch in range(epoch):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        acc = calc_acc(model)
        total_time = time.time() - start
        print(str(epoch + 1) + ' エポック目', format(total_time, ".2f") + ' 秒', '精度 ' + format(acc, ".4f"))

In [7]:
train(30)

1 エポック目 48.51 秒 精度 0.3147
2 エポック目 98.34 秒 精度 0.3579
3 エポック目 147.90 秒 精度 0.3335
4 エポック目 197.96 秒 精度 0.5554
5 エポック目 249.53 秒 精度 0.5689
6 エポック目 299.92 秒 精度 0.7350
7 エポック目 349.91 秒 精度 0.7800
8 エポック目 400.63 秒 精度 0.7270
9 エポック目 451.18 秒 精度 0.7780
10 エポック目 502.39 秒 精度 0.8216
11 エポック目 553.34 秒 精度 0.8477
12 エポック目 604.36 秒 精度 0.8477
13 エポック目 656.40 秒 精度 0.8767
14 エポック目 707.33 秒 精度 0.8784
15 エポック目 757.85 秒 精度 0.8864
16 エポック目 810.29 秒 精度 0.9000
17 エポック目 862.02 秒 精度 0.8954
18 エポック目 913.94 秒 精度 0.8996
19 エポック目 965.94 秒 精度 0.9061
20 エポック目 1018.13 秒 精度 0.9089
21 エポック目 1070.24 秒 精度 0.9100
22 エポック目 1121.12 秒 精度 0.9140
23 エポック目 1173.96 秒 精度 0.9153
24 エポック目 1226.18 秒 精度 0.9166
25 エポック目 1277.86 秒 精度 0.9186
26 エポック目 1331.17 秒 精度 0.9211
27 エポック目 1383.94 秒 精度 0.9233
28 エポック目 1435.31 秒 精度 0.9235
29 エポック目 1486.99 秒 精度 0.9255
30 エポック目 1540.73 秒 精度 0.9273


In [8]:
model = GCN(dataset.num_node_features, 32, dataset.num_classes)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

In [9]:
def train_neighborhood_sampling(epoch, batch_size=128):
    model.train()
    loader = NeighborLoader(
        data,
        num_neighbors=[5] * 2,
        batch_size=batch_size,
        input_nodes=data.train_mask,
    )
    start = time.time()
    for epoch in range(epoch):
        for sampled_data in loader:
            optimizer.zero_grad()
            out = model(sampled_data)
            loss = F.nll_loss(out[:batch_size], sampled_data.y[:batch_size])
            loss.backward()
            optimizer.step()
        acc = calc_acc(model)
        total_time = time.time() - start
        print(str(epoch + 1) + ' エポック目', format(total_time, ".2f") + ' 秒', '精度 ' + format(acc, ".4f"))

In [10]:
train_neighborhood_sampling(3)
# サンプリングがある方が 1 エポックあたりの時間が短く、かつミニバッチだと 1 エポックに多く更新するので必要なエポック数も少ない

1 エポック目 25.69 秒 精度 0.9273
2 エポック目 55.55 秒 精度 0.9322
3 エポック目 80.46 秒 精度 0.9333
